In [1]:
import time
import re, ast
import numpy as np
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'



In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
conf = SparkConf() \
    .setAppName("Streaming test") \
    .setMaster("local[2]") \
    .set("spark.cassandra.connection.host", "127.0.0.1") # "local[2]" to run locally with 2 cores
sc = SparkContext(conf=conf) 
sqlContext=SQLContext(sc)
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
ssc = StreamingContext(sc, 10) 
# 0.1 = batchDuration
# Main entry point for Spark Streaming functionality. A StreamingContext represents the
# connection to a Spark cluster, and can be used to create DStream various input sources.
# A Discretized Stream (DStream), the basic abstraction in Spark Streaming, 
# is a continuous sequence of RDDs (of the same type) representing a continuous stream of data
# DStreams can either be created from live data (such as, data from TCP sockets, Kafka, Flume, etc.) using a 
# StreamingContext or it can be generated by transforming existing DStreams using operations such as map and window.
# https://blog.jetoile.fr/2014/05/rdd-quest-ce-que-cest.html
ssc.checkpoint("checkpoint")
# Sets the context to periodically checkpoint the DStream operations for master fault-tolerance.
# Parameters:	directory – HDFS-compatible directory where the checkpoint data will be reliably stored
ssc.remember(1)
# Set each DStreams in this context to remember RDDs it generated in the last given duration. 

In [4]:
kvs = KafkaUtils.createStream(ssc, "127.0.0.1:2181", "spark-streaming-consumer", {'test2': 1})
# createStream(ssc, zkQuorum, groupId, topics)
# Parameters:	
#				ssc – StreamingContext object
#				zkQuorum – Zookeeper quorum (hostname:port,hostname:port,..).
#				groupId – The group id for this consumer.
#				topics – Dict of (topic_name -> numPartitions) to consume. Each partition is consumed in its own thread.
# Topic partitions in Kafka does not correlate to partitions of RDDs generated in Spark Streaming. 

In [5]:
def readInput(line):
    # Convert the input string into a pair of numbers
    print('now')
    vec= [float(x) for x in line.split()] 


    # The pair represents the inputs for two model (forgetting factor 1 and 0.99)
    return [('mod1',('mod1',np.array(vec))),('mod2',('mod2',np.array(vec)))]

In [6]:
def updateFunction(new_values, state): 
    ## RLS update function, formulas slide 51
    ## new_values = (key, [i, y, proxies])
    ## state = what is given by sc.parallelize in next cell on the first call
    ##       = what this function returned on the other calls
    if (len(new_values)>0 ):
        
        key=new_values[0][0]
        yx=new_values[0][1]
        i=yx[0]
        y=yx[1]
        x=yx[2:]
        n=len(x)
        
        beta=state[1]
        beta.shape=(n,1) # Transorms beta in a matrix of n rows and 1 column
        V=state[2]
        mu=state[3]
        sse=state[4]  ## sum of squared errors
        N=state[5]    ## number of treated samples
        x.shape=(1,n)
        err=y-x.dot(beta)
        sse=sse+pow(err,2.0)
        V=1.0/mu*(V-V.dot(x.T).dot(x).dot(V)/(1.0+float(x.dot(V).dot(x.T)))) # dot = matrix multiplication
                                                                             # .T = Transpose
        gamma=V.dot(x.T)
        beta=beta+gamma*err
        proxyPrediction=x.dot(beta)
        
        if (key=='mod1'):          
            return (key,beta,V,mu,sse/(N+1.0),N+1,i, proxyPrediction)  ## update formula mod1
        else:
            return (key,beta,V,mu,sse/(N+1.0),N+1,i, proxyPrediction)  ## update formula mod2
        
    else:
        return state

In [7]:
import re, ast
n=10 # number of features
beta1=np.zeros(n)  ## initial parameter vector slide 49
beta2=np.zeros(n)
V1=np.diag(np.zeros(n)+10) ## initial covariance matrix slide 49
V2=np.diag(np.zeros(n)+1)
mu1=1.0 # forgetting factor slide 50
mu2=0.99
data = kvs.map(lambda x: np.array(ast.literal_eval(x[1])))
# map: Returns a new DStream by applying a function to each element of DStream.
# literal_eval: This can be used for safely evaluating strings containing Python values (strings, numbers, tuples, lists, dicts, booleans, and None)
# from untrusted sources without the need to parse the values oneself.
data=data.flatMap(lambda x: [('mod1',('mod1',1.0*np.array(x))),
                            ('mod2',('mod2',1.0*np.array(x)))])
data.pprint() # prints received array
# Return a new DStream by applying a function to all elements of this DStream, and then flattening the results
#initialStateRDD = sc.parallelize([('k',([1,2,3]))])
initialStateRDD = sc.parallelize([(u'mod1', ('mod1',beta1,V1,mu1,0,0,0,0)),
                                  (u'mod2', ('mod2',beta2,V2,mu2,0,0,0,0))])
# The elements of the collection [] are copied to form a distributed dataset that can be operated on in parallel.
# ('mod1',beta1,V1,mu1,0,0,0)) == what updateFunction returns
data2=data.updateStateByKey(updateFunction,initialRDD=initialStateRDD)
# Return a new "state" DStream where the state for each key is updated by applying the given function on the previous
# state of the key and the new values for the key. This can be used to maintain arbitrary state data for each key.
#data.pprint()
data2.map(lambda x: [x[1][i] for i in [0,1,4,7]]).pprint() 
# prints key, beta, (residual sum of squares)/(numOfSamples+1) and proxyPrediction

In [8]:
ssc.start()


-------------------------------------------
Time: 2017-11-13 17:08:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:08:00
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:08:10
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:08:10
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:08:20
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:08:20
-------------------------------------------
['mod2', array(

-------------------------------------------
Time: 2017-11-13 17:11:40
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:11:40
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:11:50
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:11:50
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:12:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:12:00
-------------------------------------------
['mod2', array(

-------------------------------------------
Time: 2017-11-13 17:15:20
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:15:20
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:15:30
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:15:30
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:15:40
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:15:40
-------------------------------------------
['mod2', array(

-------------------------------------------
Time: 2017-11-13 17:19:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:19:00
-------------------------------------------
['mod2', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]
['mod1', array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), 0, 0]

-------------------------------------------
Time: 2017-11-13 17:19:10
-------------------------------------------
('mod1', ('mod1', array([  0.00000000e+00,   1.01049719e+00,   1.00000000e+00,
         2.02646572e-05,   3.17115195e-05,   3.34703254e-05,
         3.39927872e-05,   3.47421387e-05,   3.50472567e-05,
         3.64944956e-05,   3.70127612e-05,   3.87653780e-05])))
('mod2', ('mod2', array([  0.00000000e+00,   1.01049719e+00,   1.00000000e+00,
         2.02646572e-05,   3.17115195e-05,   3.34703254e-05,
         3.39927872e-05,   3.47421387e-05,   3.50472567e-05,
         3.64944956e-05,   3.70127612e-05,

-------------------------------------------
Time: 2017-11-13 17:20:30
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:20:30
-------------------------------------------
['mod2', array([[  6.98712308e-01],
       [  1.84714319e-05],
       [  2.28198070e-05],
       [  2.35829503e-05],
       [  2.40334748e-05],
       [  2.43897041e-05],
       [  2.50331540e-05],
       [  2.56943945e-05],
       [  2.65215183e-05],
       [  2.74589250e-05]]), array([[ 0.66495367]]), array([[ 0.69871232]])]
['mod1', array([[  9.88833113e-01],
       [  2.88775448e-05],
       [  3.27155372e-05],
       [  3.35000071e-05],
       [  3.41918372e-05],
       [  3.45897870e-05],
       [  3.57734349e-05],
       [  3.64871665e-05],
       [  3.79528014e-05],
       [  3.90972164e-05]]), array([[ 0.5214184]]), array([[ 0.98883312]])]

-------------------------------------------
Time: 2017-11-13 17:20:40
-------------------------------------------

...

-------------------------------------------
Time: 2017-11-13 17:59:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 17:59:00
-------------------------------------------
['mod2', array([[  1.03383819e+00],
       [  4.41409991e-05],
       [  4.44591973e-05],
       [  4.49220001e-05],
       [  4.54433948e-05],
       [  4.59597692e-05],
       [  4.63817675e-05],
       [  4.67391939e-05],
       [  4.69946626e-05],
       [  4.72700670e-05]]), array([[  8.39206137e-08]]), array([[ 1.03383821]])]
['mod1', array([[  1.05108564e+00],
       [  4.21965485e-05],
       [  3.87582695e-05],
       [  3.97082693e-05],
       [  4.16187750e-05],
       [  4.36196581e-05],
       [  4.49059874e-05],
       [  4.55686776e-05],
       [  4.55826378e-05],
       [  4.57872892e-05]]), array([[  7.90678237e-06]]), array([[ 1.05108565]])]

-------------------------------------------
Time: 2017-11-13 17:59:10
--------------------------------

-------------------------------------------
Time: 2017-11-13 18:00:20
-------------------------------------------
['mod2', array([[  1.03205917e+00],
       [  4.36270112e-05],
       [  4.39942498e-05],
       [  4.44881088e-05],
       [  4.50337317e-05],
       [  4.55733288e-05],
       [  4.60221159e-05],
       [  4.64057291e-05],
       [  4.66817921e-05],
       [  4.69741337e-05]]), array([[  2.41016261e-05]]), array([[ 1.03205919]])]
['mod1', array([[  1.04881731e+00],
       [  3.71200346e-05],
       [  3.42701612e-05],
       [  3.55760530e-05],
       [  3.77690067e-05],
       [  4.00408131e-05],
       [  4.16340425e-05],
       [  4.25988251e-05],
       [  4.28542198e-05],
       [  4.32625042e-05]]), array([[  5.67357334e-05]]), array([[ 1.04881733]])]

-------------------------------------------
Time: 2017-11-13 18:00:30
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 18:00:30
--------------------------------

-------------------------------------------
Time: 2017-11-13 18:01:50
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 18:01:50
-------------------------------------------
['mod2', array([[  1.03211262e+00],
       [  4.36426489e-05],
       [  4.40082771e-05],
       [  4.45014683e-05],
       [  4.50462092e-05],
       [  4.55849463e-05],
       [  4.60329231e-05],
       [  4.64159188e-05],
       [  4.66914423e-05],
       [  4.69832390e-05]]), array([[  6.32029327e-07]]), array([[ 1.03211264]])]
['mod1', array([[  1.04848097e+00],
       [  3.63575487e-05],
       [  3.36017561e-05],
       [  3.49472694e-05],
       [  3.71896281e-05],
       [  3.95098075e-05],
       [  4.11488216e-05],
       [  4.21499722e-05],
       [  4.24371542e-05],
       [  4.28774215e-05]]), array([[  6.28958904e-06]]), array([[ 1.04848099]])]

-------------------------------------------
Time: 2017-11-13 18:02:00
--------------------------------

-------------------------------------------
Time: 2017-11-13 18:03:20
-------------------------------------------
('mod1', ('mod1', array([  4.50000000e+01,   1.00596853e+00,   1.00000000e+00,
         4.83700566e-05,   4.83749463e-05,   4.83749463e-05,
         4.83754147e-05,   4.84337745e-05,   4.84915088e-05,
         4.85372082e-05,   4.89137738e-05,   4.89614396e-05])))
('mod2', ('mod2', array([  4.50000000e+01,   1.00596853e+00,   1.00000000e+00,
         4.83700566e-05,   4.83749463e-05,   4.83749463e-05,
         4.83754147e-05,   4.84337745e-05,   4.84915088e-05,
         4.85372082e-05,   4.89137738e-05,   4.89614396e-05])))

-------------------------------------------
Time: 2017-11-13 18:03:20
-------------------------------------------
['mod2', array([[  1.03194040e+00],
       [  4.35876962e-05],
       [  4.39618114e-05],
       [  4.44583767e-05],
       [  4.50059871e-05],
       [  4.55451026e-05],
       [  4.59951572e-05],
       [  4.63806449e-05],
       [  4.6644

-------------------------------------------
Time: 2017-11-13 18:04:40
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 18:04:40
-------------------------------------------
['mod2', array([[  1.03114387e+00],
       [  4.33493315e-05],
       [  4.37476120e-05],
       [  4.42590522e-05],
       [  4.48172728e-05],
       [  4.53665556e-05],
       [  4.58266973e-05],
       [  4.62063677e-05],
       [  4.64789976e-05],
       [  4.67880006e-05]]), array([[  1.97184614e-05]]), array([[ 1.03114389]])]
['mod1', array([[  1.04666193e+00],
       [  3.21364722e-05],
       [  2.98938766e-05],
       [  3.15424679e-05],
       [  3.40295499e-05],
       [  3.65616969e-05],
       [  3.84078293e-05],
       [  3.94678953e-05],
       [  3.98011239e-05],
       [  4.03984743e-05]]), array([[  4.56257832e-05]]), array([[ 1.04666195]])]

-------------------------------------------
Time: 2017-11-13 18:04:50
--------------------------------

In [9]:
ssc.stop(stopSparkContext=False,stopGraceFully=True)

-------------------------------------------
Time: 2017-11-13 18:05:00
-------------------------------------------

-------------------------------------------
Time: 2017-11-13 18:05:00
-------------------------------------------
['mod2', array([[  1.03114387e+00],
       [  4.33493315e-05],
       [  4.37476120e-05],
       [  4.42590522e-05],
       [  4.48172728e-05],
       [  4.53665556e-05],
       [  4.58266973e-05],
       [  4.62063677e-05],
       [  4.64789976e-05],
       [  4.67880006e-05]]), array([[  1.97184614e-05]]), array([[ 1.03114389]])]
['mod1', array([[  1.04666193e+00],
       [  3.21364722e-05],
       [  2.98938766e-05],
       [  3.15424679e-05],
       [  3.40295499e-05],
       [  3.65616969e-05],
       [  3.84078293e-05],
       [  3.94678953e-05],
       [  3.98011239e-05],
       [  4.03984743e-05]]), array([[  4.56257832e-05]]), array([[ 1.04666195]])]

-------------------------------------------
Time: 2017-11-13 18:05:10
--------------------------------

In [ ]:
a=[1,2,3,4]


In [ ]:
[a[i] for i in [1,0,2,3]]
